In [ ]:
/*
 * The first step when defining a typeclass consists on
 * defining the trait it implements.
 */
trait Sized[A] {
    def sized(a: A): Int
}


/* 
 * The second (and the last!) step when defining a typeclass consists on
 * defining a companion object for the typeclass trait giving some useful default implementations.
 *
 * These useful default implementations are provided as implicit objects and/or implicit functions, which
 * can be found after local implicits so that you can still override the default implementations if you wish.
 *
 * See more about implicit search order here:
 * https://stackoverflow.com/questions/5598085/where-does-scala-look-for-implicits/5598107#5598107
 */
object Sized {
  implicit object IntSized    extends Sized[Int]    { def sized(a: Int) = a }
  implicit object StringSized extends Sized[String] { def sized(a: String) = a.length }

  /* This one is more elaborated: Sized[List[A]] needs Sized[A], which can be found implicitly. */
  implicit def ListSized[A](implicit itemSized: Sized[A]) = new Sized[List[A]] {
    def sized(a: List[A]): Int = a.map(item => itemSized.sized(item)).sum
  } 
}

In [ ]:
// This is a simple example which uses the typeclass

trait ExampleSize {
    def size[A](a: A)(implicit s: Sized[A]): Int = s.sized(a)
}
object Example1 extends ExampleSize {
    def main = {
      println(size(2)) // prints 2
      println(size("abc")) // prints 3
      println(size(List("abc", "d", "efgh"))) // prints 8
    }
}
Example1.main

In [ ]:
// This example supplies an implicit implementation

object Example2 extends ExampleSize {
    implicit object BooleanSize extends Sized[Boolean] { def sized(a: Boolean) = if(a) 1 else 0 }
    def main = {
      println(size(false)) // prints 0
      println(size(true))  // prints 1
    }
}
Example2.main

In [ ]:
// This example supplies an implicit implementation which is used by a default implementation

object Example3 extends ExampleSize {
    implicit object BooleanSize     extends Sized[Boolean]       { def sized(a: Boolean) = if(a) 1 else 0 }
    def main = {
      println(size(List(true, false, true, false, true))) // prints 3
    }
}
Example3.main